In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import sys

sys.path.append(os.path.abspath("../"))
from dataset import DS_NAMES
from dataset import DS_DIMENSIONS
from utilities.figure_tools import get_latest_results_directory, make_fig_directory, load_results

results_path = "../results/vary_enum.csv"
output_dir = "./vary_enum/"
export_figures = False

if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)


In [3]:
all_results = pd.read_csv(results_path).groupby(["dataset", "explainer", "hard_voting", "enumeration"]).mean().reset_index()
print("datasets", list(all_results["dataset"].unique()))
all_results.head()

datasets ['cancer', 'glass', 'magic', 'spambase', 'vertebral']


,dataset,explainer,hard_voting,enumeration,n_trees,max_depth,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain
0,cancer,FACETIndex,False,Axes,100.0,NaN,0.0,0.947368,0.95122,0.906977,0.928571,1.0,0.334741,14.50,24.542435,2.495010,0.124750,20.0
1,cancer,FACETIndex,False,Probability,100.0,NaN,0.0,0.947368,0.95122,0.906977,0.928571,1.0,0.334017,14.35,15.870547,2.265010,0.113250,20.0
2,cancer,FACETIndex,False,Size,100.0,NaN,0.0,0.947368,0.95122,0.906977,0.928571,1.0,0.340140,14.45,34.932947,2.505071,0.125254,20.0
3,cancer,FACETIndex,True,Axes,100.0,NaN,0.0,0.947368,0.95122,0.906977,0.928571,1.0,0.337977,14.70,18.100935,2.557876,0.127894,20.0
4,cancer,FACETIndex,True,Probability,100.0,NaN,0.0,0.947368,0.95122,0.906977,0.928571,1.0,0.337161,14.60,14.371069,2.402060,0.120103,20.0


In [7]:
all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]]

,dataset,hard_voting,enumeration,avg_dist,avg_length,prep_time
0,cancer,False,Axes,0.334741,14.50,24.542435
1,cancer,False,Probability,0.334017,14.35,15.870547
2,cancer,False,Size,0.340140,14.45,34.932947
3,cancer,True,Axes,0.337977,14.70,18.100935
4,cancer,True,Probability,0.337161,14.60,14.371069
5,cancer,True,Size,0.340606,14.45,30.524574
6,glass,False,Axes,0.199991,5.40,16.976023
7,glass,False,Probability,0.220523,6.15,8.742533
8,glass,False,Size,0.227062,6.40,29.065061
9,glass,True,Axes,0.200304,5.50,11.214154


In [107]:
idx = all_results.groupby(["explainer", "dataset"])["avg_dist"].idxmin()
all_results.loc[idx, ["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].head(100)

,dataset,hard_voting,enumeration,avg_dist,avg_length,prep_time
1,cancer,False,Probability,0.334017,14.35,15.870547
6,glass,False,Axes,0.199991,5.40,16.976023
12,magic,False,Axes,0.165568,8.15,24.704540
18,spambase,False,Axes,0.088125,18.25,34.285959
24,vertebral,False,Axes,0.072076,3.55,16.747011


In [64]:
idx = all_results.groupby(["explainer", "dataset", "hard_voting"])["avg_dist"].idxmin()
all_results.loc[idx, ["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].head(100)

,dataset,hard_voting,enumeration,avg_dist,avg_length,prep_time
1,cancer,False,Probability,0.334017,14.35,15.870547
4,cancer,True,Probability,0.337161,14.60,14.371069
6,glass,False,Axes,0.199991,5.40,16.976023
9,glass,True,Axes,0.200304,5.50,11.214154
12,magic,False,Axes,0.165568,8.15,24.704540
15,magic,True,Axes,0.165659,8.20,18.927443
18,spambase,False,Axes,0.088125,18.25,34.285959
21,spambase,True,Axes,0.088804,18.45,27.311240
24,vertebral,False,Axes,0.072076,3.55,16.747011
27,vertebral,True,Axes,0.072110,3.55,11.010783


In [22]:
all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]]

,dataset,hard_voting,enumeration,avg_dist,avg_length,prep_time
0,cancer,False,Axes,0.334741,14.50,24.542435
1,cancer,False,Probability,0.334017,14.35,15.870547
2,cancer,False,Size,0.340140,14.45,34.932947
3,cancer,True,Axes,0.337977,14.70,18.100935
4,cancer,True,Probability,0.337161,14.60,14.371069
5,cancer,True,Size,0.340606,14.45,30.524574
6,glass,False,Axes,0.199991,5.40,16.976023
7,glass,False,Probability,0.220523,6.15,8.742533
8,glass,False,Size,0.227062,6.40,29.065061
9,glass,True,Axes,0.200304,5.50,11.214154


In [29]:
datasets = all_results["dataset"].unique()
votings = all_results["hard_voting"].unique()
enums = all_results["enumeration"].unique()

df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]]
df = df.pivot(index=["dataset", "enumeration"], columns=["hard_voting"])
# df = pd.DataFrame(, index=pd.MultiIndex.from_product([datasets, votings, enums]))
df.head(100)

avg_dist           avg_length         prep_time  \
hard_voting               False      True      False   True      False   
dataset   enumeration                                                    
cancer    Axes         0.334741  0.337977      14.50  14.70  24.542435   
          Probability  0.334017  0.337161      14.35  14.60  15.870547   
          Size         0.340140  0.340606      14.45  14.45  34.932947   
glass     Axes         0.199991  0.200304       5.40   5.50  16.976023   
          Probability  0.220523  0.221467       6.15   6.20   8.742533   
          Size         0.227062  0.227062       6.40   6.40  29.065061   
magic     Axes         0.165568  0.165659       8.15   8.20  24.704540   
          Probability  0.170170  0.170117       8.85   8.80  16.948613   
          Size         0.170573  0.170584       8.85   8.90  37.466001   
spambase  Axes         0.088125  0.088804      18.25  18.45  34.285959   
          Probability  0.096883  0.096550      18.10  18.45  26.695983   
          Size         0.097555  0.097721      18.05  18.25  44.147446   
vertebral Axes         0.072076  0.072110       3.55   3.55  16.747011   
          Probability  0.073067  0.072915       3.70   3.70   8.665000   
          Size         0.073155  0.073158       4.10   4.10  29.019443   

                                  
hard_voting                 True  
dataset   enumeration             
cancer    Axes         18.100935  
          Probability  14.371069  
          Size         30.524574  
glass     Axes         11.214154  
          Probability   7.278757  
          Size         24.684381  
magic     Axes         18.927443  
          Probability  14.686814  
          Size         32.980028  
spambase  Axes         27.311240  
          Probability  24.144609  
          Size         37.975677  
vertebral Axes         11.010783  
          Probability   7.393135  
          Size         23.915154

In [41]:
df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].set_index(["dataset", "hard_voting", "enumeration"])
df.head()

avg_dist  avg_length  prep_time
dataset hard_voting enumeration                                 
cancer  False       Axes         0.334741       14.50  24.542435
                    Probability  0.334017       14.35  15.870547
                    Size         0.340140       14.45  34.932947
        True        Axes         0.337977       14.70  18.100935
                    Probability  0.337161       14.60  14.371069

In [55]:
df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].set_index(["hard_voting", "dataset", "enumeration"])
diff = df.loc[False,:,:]["avg_dist"] / df.loc[True,:,:]["avg_dist"]
diff.head(100)

dataset    enumeration
cancer     Axes           0.990427
           Probability    0.990675
           Size           0.998630
glass      Axes           0.998437
           Probability    0.995737
           Size           1.000000
magic      Axes           0.999446
           Probability    1.000310
           Size           0.999931
spambase   Axes           0.992361
           Probability    1.003447
           Size           0.998300
vertebral  Axes           0.999521
           Probability    1.002079
           Size           0.999966
Name: avg_dist, dtype: float64

In [106]:
df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].set_index(["hard_voting", "dataset", "enumeration"])
df = df.loc[False,:,:].reset_index()
mins = df.groupby(["dataset"])["avg_dist"].min()
df = df.set_index(["dataset", "enumeration"])["avg_dist"] / mins
df.head(100)
# df[["dataset", "enumeration", "avg_dist"]].groupby(["dataset", "enumeration"]) / mins
# df = all_results.groupby(["explainer", "dataset", "hard_voting"])["avg_dist"].min()
# all_results.loc[idx, ["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].head(100)

dataset    enumeration
cancer     Axes           1.002169
           Probability    1.000000
           Size           1.018331
glass      Axes           1.000000
           Probability    1.102663
           Size           1.135360
magic      Axes           1.000000
           Probability    1.027796
           Size           1.030229
spambase   Axes           1.000000
           Probability    1.099376
           Size           1.107006
vertebral  Axes           1.000000
           Probability    1.013753
           Size           1.014982
Name: avg_dist, dtype: float64

In [56]:
enum_comp = df.loc[False,:,:]["avg_dist"]
enum_comp.head(100)

dataset    enumeration
cancer     Axes           0.334741
           Probability    0.334017
           Size           0.340140
glass      Axes           0.199991
           Probability    0.220523
           Size           0.227062
magic      Axes           0.165568
           Probability    0.170170
           Size           0.170573
spambase   Axes           0.088125
           Probability    0.096883
           Size           0.097555
vertebral  Axes           0.072076
           Probability    0.073067
           Size           0.073155
Name: avg_dist, dtype: float64

ds: cancer, enum: Axes, hard-soft: 0.003235322609509872
ds: cancer, enum: Probability, hard-soft: 0.00314414648596828
ds: cancer, enum: Size, hard-soft: 0.0004665221096812888
ds: glass, enum: Axes, hard-soft: 0.0003131748118953004
ds: glass, enum: Probability, hard-soft: 0.000944138563838709
ds: glass, enum: Size, hard-soft: 0.0
ds: magic, enum: Axes, hard-soft: 9.184607771628439e-05
ds: magic, enum: Probability, hard-soft: -5.2702417714711736e-05
ds: magic, enum: Size, hard-soft: 1.181401641051072e-05
ds: spambase, enum: Axes, hard-soft: 0.0006783338872494993
ds: spambase, enum: Probability, hard-soft: -0.00033281111304049593
ds: spambase, enum: Size, hard-soft: 0.00016610879487040375
ds: vertebral, enum: Axes, hard-soft: 3.4520391881801005e-05
ds: vertebral, enum: Probability, hard-soft: -0.0001515624791944975
ds: vertebral, enum: Size, hard-soft: 2.4676515291005563e-06


In [ ]:
df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].set_index(["dataset", "enumeration", "hard_voting"])
for ds in all_results["dataset"].unique():
    for enum in all_results["enumeration"].unique():
        soft_dist = df.loc[ds, enum, False]["avg_dist"]
        hard_dist = df.loc[ds, enum, True]["avg_dist"]
        print("ds: {}, enum: {}, hard-soft: {}".format(ds, enum, hard_dist-soft_dist))

In [4]:
cmap = plt.get_cmap("tab10")
colors = cmap.colors
ds_colors = {
    "cancer": colors[0],
    "glass": colors[1],
    "magic": colors[2],
    "spambase": colors[3],
    "vertebral": colors[4]
}
ds_markers = {
    "cancer": "^",
    "glass": "o",
    "magic": "v",
    "spambase": "s",
    "vertebral": "D"
}
marker_size = 8
line_width = 2
marker_sizes = {
    "cancer": marker_size,
    "glass": marker_size,
    "magic": marker_size,
    "spambase": marker_size,
    "vertebral": marker_size
}

In [5]:
def render_plot(xfeature, yfeature, xlabel, ylabel, title, save_name=None, xrange=None, yrange=None, xlog=False, ylog=False, lloc=None, loc="best", lhorz=False):

    fig = plt.figure(figsize=[6.4, 4.8], frameon=True)
    ax = fig.add_subplot(1, 1, 1)
    for ds in all_results["dataset"].unique():
        is_ds = all_results["dataset"] == ds
        for expl in all_results["explainer"].unique():
            is_expl = all_results["explainer"] == expl
            matching_rows = is_ds & is_expl
            ax.plot(all_results[matching_rows][xfeature], all_results[matching_rows][yfeature], label=ds, lw=line_width, marker=ds_markers[ds], ms=marker_sizes[ds], color=ds_colors[ds])

        # title and axis labels
        if title is not None:
            plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        # legend
        if lhorz:
            plt.legend(loc=loc, ncol=len(all_results["dataset"].unique()), bbox_to_anchor=(0, 1.09, 1, 0), handletextpad=0.5, columnspacing=1, handlelength=1.5)
        else:
            plt.legend(loc=loc)
        # x and y ranges
        if xrange is not None:
            plt.xlim(xrange)
        if yrange is not None:
            plt.ylim(yrange)
        # x and y scales
        if ylog:
            plt.yscale('log')
        if xlog:
            plt.xscale('log')
        plt.subplots_adjust(top=1.0)
        
        if export_figures and save_name is not None:
            plt.savefig(output_dir + save_name, bbox_inches="tight")

In [6]:
all_results["n_rects"].unique()

array([   100,   1000,   5000,  10000,  20000,  30000,  40000,  50000,
        60000,  70000,  80000,  90000, 100000], dtype=int64)

In [7]:
render_plot(xfeature="n_rects", yfeature="avg_dist", xlabel="Number of Hyperrectangles", ylabel="Explanation Distance", title=None, save_name="dist_vs_rects.png", loc="upper center", lhorz=True)

In [8]:
render_plot(xfeature="n_rects", yfeature="prep_time", xlabel="Number of Rectangles", ylabel="Initialization Time (sec)", title=None, save_name="prep_vs_rects.png", loc="upper center", lhorz=True)

In [9]:
render_plot(xfeature="n_rects", yfeature="sample_time", xlabel="Number of Rectangles", ylabel="Sample Explanation Time (sec)", title=None, save_name="explain_vs_nrects", loc="upper center", lhorz=True)